# Cyber Bullying

In [9]:
import os
import nltk
nltk.download('stopwords')
import pandas as pd
import numpy as np
import tensorflow.compat.v1 as tf
import re
tf.disable_v2_behavior()
from autocorrect import spell
import string 

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\eviat\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [4]:
from nltk.corpus import wordnet
a = wordnet.synsets('ass')
a

[Synset('buttocks.n.01'),
 Synset('ass.n.02'),
 Synset('ass.n.03'),
 Synset('fuck.n.01'),
 Synset('arsenic.n.02'),
 Synset('american_samoa.n.01')]

In [8]:
from nltk.corpus import wordnet
syn = wordnet.synsets('fuck')[0]
print ("Synset name :  ", syn.name())
# Defining the word
print ("\nSynset meaning : ", syn.definition())
# list of phrases that use the word in context
print ("\nSynset example : ", syn)

Synset name :   fuck.n.01

Synset meaning :  slang for sexual intercourse

Synset example :  []


## Data cleanning

In [10]:
df = pd.read_csv("./Data/Source1/youtube_parsed_dataset.csv")
df

,index,UserIndex,Text,Number of Comments,Number of Subscribers,Membership Duration,Number of Uploads,Profanity in UserID,Age,oh_label
0,0,X1,Does N.e.bodyelse Hear her Crazy ass Screamin ...,10,1,3,3,0,15,0
1,1,X2,There are so many things that are incorrect wi...,3,0,6,5,0,31,0
2,2,X3,3:26 hahah my boyfriend showed this song to me...,7,0,3,5,0,43,1
3,3,X2218,dick beyonce fuck y a ass hole you are truely ...,34,0,3,5,0,44,1
4,4,X5,DongHaeTaemin and Kai ;A; luhansehun and bacon...,11,173,5,5,0,21,0
...,...,...,...,...,...,...,...,...,...,...
3459,3464,X3465,She pretty I love this song I miss the old kel...,15,2,4,7,0,23,1
3460,3465,X3466,Status-Online Im ZxkillergirlzX! I'm Zxkillerg...,4,28,4,23,1,15,0
3461,3466,X3467,JR so cute EXO M Better I agree like yeah yeah...,23,0,5,3,0,33,0
3462,3467,X3468,! !,5,0,6,5,0,38,0


In [11]:
df.drop(['index',"UserIndex","Number of Comments","Number of Subscribers","Membership Duration","Number of Uploads","Profanity in UserID","Age"], axis=1, inplace=True)
df

,Text,oh_label
0,Does N.e.bodyelse Hear her Crazy ass Screamin ...,0
1,There are so many things that are incorrect wi...,0
2,3:26 hahah my boyfriend showed this song to me...,1
3,dick beyonce fuck y a ass hole you are truely ...,1
4,DongHaeTaemin and Kai ;A; luhansehun and bacon...,0
...,...,...
3459,She pretty I love this song I miss the old kel...,1
3460,Status-Online Im ZxkillergirlzX! I'm Zxkillerg...,0
3461,JR so cute EXO M Better I agree like yeah yeah...,0
3462,! !,0


In [12]:
df["Text"][1],df["oh_label"][1],

("There are so many things that are incorrect with your comment it's unbelievable. Guns don't kill people. A gun doesn't get up off a table and then shoot someone. He's not the reason soldiers are at war he's merely showing us weapons and explaining how they work and whether he likes them or not. You blame him for violent video games...Why would this guy mess around with video games when he has real weapons he can use and practice with. I can't say much more coz I don't have more space. Retard. This is all we need an Australian version of 1 Direction.... my dick was bleeding from how hard I was masturbating to this",
 0)

In [13]:
from spellchecker import SpellChecker
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
nltk.download('words')
nltk.download('wordnet')
words = set(nltk.corpus.words.words())
spell = SpellChecker()
lemmatizer = WordNetLemmatizer()

def process_row(row):
    row = row.translate(str.maketrans('', '', string.punctuation))
    row = re.sub('\w*\d\w*', ' ', row) #remove words with numbers
    row = re.sub('[‘’“”…]', ' ', row)
    row = row.lower()
    stop_words = stopwords.words('english')
    row = " ".join(lemmatizer.lemmatize(w) for w in nltk.wordpunct_tokenize(row)) # spell checker
    row = " ".join(spell.correction(w) for w in nltk.wordpunct_tokenize(row)) # spell checker
    return " ".join(w for w in nltk.wordpunct_tokenize(row) if w in words and w not in stop_words) # word exist in corpus and remove stop words 

[nltk_data] Downloading package words to
[nltk_data]     C:\Users\eviat\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\eviat\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [14]:
df["Text"] = df["Text"].apply(process_row)
df

,Text,oh_label
0,doe hear crazy hoe say stupid hoe nobody see b...,0
1,many thing incorrect comment unbelievable gun ...,0
2,song love seizure u corgi training want rape,1
3,dick hole dog bitch look like dick god make si...,1
4,kai bacon ad kai henry dabba love rapping yeah...,0
...,...,...
3459,pretty love song miss old kelly yes love song ...,1
3460,band band practice,0
3461,cute better agree like yeah yeah yeah wow wow ...,0
3462,,0


In [15]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer()
data_cv = cv.fit_transform(df.Text)
data_cv = pd.DataFrame(data_cv.toarray(), columns=cv.get_feature_names())
data_cv["oh_label"] = df["oh_label"]
data_cv


,abandon,abandoned,abase,abate,abb,abbas,abbey,abbreviation,abdicate,abdomen,...,zipper,zo,zombie,zone,zoned,zoning,zoo,zoom,zucchini,oh_label
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3459,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3460,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3461,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3462,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [16]:
data_cv.to_csv("./Data/Source1/Youtube_Clean.csv")

In [19]:
cleaner = pd.read_csv("./Data/Source1/Youtube_Clean.csv")

In [49]:
res = cleaner.sum(axis=0)
res = res[res > res.median()]
ls = res.index.to_list()
del ls[0]
ls
pandas_df = cleaner[cleaner.columns.intersection(ls)]
pandas_df

,abandoned,ability,able,abomination,abortion,absolute,absolutely,absorbed,abuse,abusive,...,zeppelin,zero,zest,zip,zo,zombie,zone,zoo,zoom,oh_label
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3459,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3460,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3461,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3462,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [17]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(pandas_df.drop('oh_label', axis=1), pandas_df["oh_label"], test_size=0.33, random_state=42)

AttributeError: 'NoneType' object has no attribute 'drop'

In [ ]:
from sklearn.linear_model import LogisticRegression
logmodel = LogisticRegression()
logmodel.fit(X_train,y_train)
predictions = logmodel.predict(X_test)

In [10]:
from sklearn.metrics import classification_report
print(classification_report(y_test,predictions))

              precision    recall  f1-score   support

           0       0.88      0.95      0.91       998
           1       0.25      0.11      0.15       146

    accuracy                           0.84      1144
   macro avg       0.56      0.53      0.53      1144
weighted avg       0.80      0.84      0.82      1144



## Basic FF NN Model

## RNN Models